In [ ]:
from numba import guvectorize, float64, float32, int16, int32
import numpy

In [ ]:
def restrict_2d(vF, vC):
    '''
    2D full weighting restriction. Only applicable when nx=ny and dx=dy.
    
    Parameters
    ----------
    vF: nF x nF array, the array on the fine grid
    vC: nC x nC array, the array on the coarse grid
    
    Return
    ----------
    vC: nC x nC array, the array on the coarse grid
    '''

    vC[1:-1, 1:-1] = (vF[1:-3:2, 1:-3:2] + vF[1:-3:2, 3:-1:2] +\
                        vF[3:-1:2, 1:-3:2] + vF[3:-1:2, 3:-1:2] +\
                        2. * (vF[2:-2:2, 1:-3:2] + vF[2:-2:2, 3:-1:2] +\
                        vF[1:-3:2, 2:-2:2] + vF[3:-1:2, 2:-2:2]) +\
                        4. * vF[2:-2:2, 2:-2:2]) / 16.0
    
    return vC

In [ ]:
@guvectorize([(int16[:,:], int16[:], int16[:,:]),
              (int32[:,:], int32[:], int32[:,:]),
              (float32[:,:], float32[:], float32[:,:]), 
              (float64[:,:], float64[:], float64[:,:])], '(n,n),(m)->(m,m)')
def restrict_2d_gvec(fine, size, coarse):
    
    J, I = fine.shape
    for j in range(2, J - 2 , 2):
        for i in range(2, I - 2, 2):
            coarse[j//2, i//2] = (1/16 * (
                        fine[j + 1, i + 1] + 
                        fine[j + 1, i - 1] +
                        fine[j - 1, i + 1] +
                        fine[j - 1, i - 1]) + 
                    1/8 * (
                    fine[j, i + 1] +
                    fine[j, i - 1] + 
                    fine[j - 1, i] + 
                    fine[j + 1, i]) +
                    1/4 * fine[j, i])
                    
    #return coarse
    

In [ ]:
from matplotlib import pyplot, cm
%matplotlib inline

In [ ]:
from matplotlib import cbook

In [ ]:
filename = cbook.get_sample_data('jacksboro_fault_dem.npz', asfileobj=False)
with numpy.load(filename) as dem:
    z = dem['elevation']

In [ ]:
z = z[:,:344]

In [ ]:
z.dtype

In [ ]:
z = numpy.float32(z)
z.dtype

In [ ]:
%%timeit 
assert z.shape[0] == 344
y = numpy.zeros_like(z[::2,::2])
restrict_2d(z, y)

In [ ]:
%%timeit
assert z.shape[0] == 344
y = numpy.zeros_like(z[::2,::2])
restrict_2d_gvec(z, numpy.empty(y.shape[0]), y)

In [ ]:
pyplot.figure(figsize=(10,10))
pyplot.imshow(z, interpolation='none', cmap=cm.viridis)

In [ ]:
%%timeit
y = numpy.zeros_like(z[::2,::2])
restrict_2d_gvec(z, numpy.empty(y.shape[0], dtype=numpy.int16), y)

In [ ]:
pyplot.figure(figsize=(10,10))
y = numpy.zeros_like(z[::2,::2])
restrict_2d_gvec(z, numpy.empty(y.shape[0], dtype=numpy.int16), y)
pyplot.imshow(y, interpolation='none', cmap=cm.viridis)
z = y.copy()

In [ ]:
def interpolation_2d(vC, vF):
    '''
    2D interpolation. Only applicable when nx=ny and dx=dy.
    
    Parameters
    ----------
    vC: 2D array, the array on the coarse grid
    vF: 2D array, the array on the fine grid
    
    Return
    ----------
    vF: 2D array, the array on the fine grid
    '''
    vF[::2, ::2] = vC[:, :]
    vF[1:-1:2, ::2] = 0.5 * (vC[:-1, :] + vC[1:, :])
    vF[::2, 1:-1:2] = 0.5 * (vC[:, :-1] + vC[:, 1:])
    vF[1:-1:2, 1:-1:2] = 0.25 * (vC[:-1, :-1] +
                                 vC[1:, :-1] + 
                                 vC[:-1, 1:] + 
                                 vC[1:, 1:])
    
    return vF

In [ ]:
@guvectorize([(int16[:,:], int16[:], int16[:,:]), 
              (float32[:,:], float32[:], float32[:,:]), 
              (float64[:,:], float64[:], float64[:,:])], '(n,n),(m)->(m,m)')
def interpolate_2d_gvec(coarse, size, fine):
    J, I = coarse.shape
    for j in range(1, J - 1):
        for i in range(1, I - 1):
            fine[2 * j, 2 * i] = coarse[j, i]
            fine[2 * j + 1, 2 * i] = .5 * (coarse[j, i] + 
                                            coarse[j + 1, i])
            fine[2 * j, 2 * i + 1] = .5 * (coarse[j, i] +
                                            coarse[j, i + 1])
            fine[2 * j + 1, 2 * i + 1] = .25 * (coarse[j, i] + 
                                                coarse[j, i + 1] + 
                                                coarse[j + 1, i] + 
                                                coarse[j + 1, i + 1])

In [ ]:
x = numpy.zeros((y.shape[0]*2, y.shape[1]*2))
interpolate_2d_gvec(y, numpy.empty(x.shape[0]), x)
y = x.copy()

In [ ]:
pyplot.figure(figsize=(8,8))
pyplot.imshow(x, interpolation='none', cmap=cm.viridis)

In [ ]:
%%timeit
x = numpy.zeros((y.shape[0]*2, y.shape[1]*2))
interpolate_2d_gvec(y, numpy.empty(x.shape[0]), x)

In [ ]:
%%timeit
x = numpy.zeros((y.shape[0]*2, y.shape[1]*2))
interpolate_2d(y, x)

## Larger arrays

In [ ]:
z = numpy.random.random((10000, 10000))

In [ ]:
%%timeit 
y = numpy.zeros_like(z[::2,::2])
restrict_2d(z, y)

In [ ]:
%%timeit
y = numpy.zeros_like(z[::2,::2])
restrict_2d_gvec(z, numpy.empty(y.shape[0]), y)

## Dask

In [ ]:
filename = cbook.get_sample_data('jacksboro_fault_dem.npz', asfileobj=False)
with numpy.load(filename) as dem:
    z = dem['elevation']

In [ ]:
z = numpy.float64(z)

In [ ]:
z = z[:300,:300]

In [ ]:
import dask.array as da

In [ ]:
#create dask array from z
d = da.from_array(z, chunks=(z.shape[0]/3, z.shape[0]/3))

In [ ]:
#ghost all chunk boundaries by 2 
#because you have to halve them later
g = da.ghost.ghost(d, depth={0: 2, 1: 2}, 
                  boundary={0: 'reflect', 1: 'reflect'})

#function to map onto blocks
def restrict_func(block):
    y = numpy.zeros_like(block[::2,::2])
    return restrict_2d_gvec(block, numpy.empty(y.shape[0]), y)

#map blocks, specify new chunksize = old chunksize / 2
g2 = g.map_blocks(restrict_func, chunks=g.chunks[0][:2])

#trim off remaining ghosting
res = da.ghost.trim_internal(g2, {0: 1, 1: 1})

a = res.compute()

In [ ]:
a.shape

In [ ]:
pyplot.figure(figsize=(12,12))
pyplot.subplot(1,2,1)
pyplot.imshow(a, interpolation='none', cmap=cm.viridis)
pyplot.subplot(1,2,2)
pyplot.imshow(z, interpolation='none', cmap=cm.viridis)

In [ ]:
x = da.random.random?

In [ ]:
x = da.random.random

In [ ]:
x = da.random.random(size=(50000, 50000), chunks=(5000, 5000))*100

In [ ]:
#ghost all chunk boundaries by 2 
#because you have to halve them later
g = da.ghost.ghost(x, depth={0: 2, 1: 2}, 
                  boundary={0: 'reflect', 1: 'reflect'})

#function to map onto blocks
def restrict_func(block):
    y = numpy.zeros_like(block[::2,::2])
    return restrict_2d_gvec(block, numpy.empty(y.shape[0]), y)

#map blocks, specify new chunksize = old chunksize / 2
g2 = g.map_blocks(restrict_func, chunks=g.chunks[0][:2])

#trim off remaining ghosting
res = da.ghost.trim_internal(g2, {0: 1, 1: 1})

In [ ]:
a = res.compute()

In [ ]:
a.mean()

In [ ]:
x.mean().compute()

In [ ]:
pyplot.imshow(a, cmap=cm.viridis)

In [ ]:
pyplot.imshow(numpy.array(x), cmap=cm.viridis)